<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=14681837495a2e1f67ae19b91f70e1ccf8804beacf3167d882580967edfa154d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 14:22:33
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -2.69 K (-0.02%)
Current PnL: -16.82 L (-11.31%)
CY Booked + Current PnL: -5.38 L (-3.62%)
-------------------
Total profit:  2.07 L
Total loss:  -18.88 L
-------------------
Total Booked + Current PnL: 21.60 L (17.75%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.33 L (56.96%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.49,196250.0,31580.0,11755.0,0.08,19.18,5.99,26.31,89.0,2.69,1.44,28.69,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.56,52.0,M-SC,3.71,87246.0,-13531.0,13628.0,-1.13,-13.43,15.62,0.09,245.0,-0.99,0.64,14.62,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,64.0,H-MC,7.93,111729.0,11225.0,17877.0,-0.23,11.17,16.00,28.96,79.0,0.63,0.82,48.93,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,58.0,H-LC,4.96,220147.0,9781.0,18999.0,-0.27,4.65,8.63,13.68,37.0,0.51,1.61,21.94,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-14.99,78.0,H-SC,12.43,227618.0,-43143.0,373089.0,6.32,-15.93,163.91,121.86,133.0,-0.12,1.67,78.06,AR,ATH,IT
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.06,159294.0,18454.0,26045.0,2.57,13.10,16.35,31.59,66.0,0.71,1.17,43.06,XY24,NTT,BANKS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85494.0,-16716.0,110108.0,2.37,-16.35,128.79,91.37,208.0,-0.15,0.63,38.78,XY25,NTT,FINANCE
59,SAIL,228.00,48.36,70.0,M-MC,7.54,233538.0,8576.0,159086.0,1.83,3.81,68.12,74.53,192.0,0.05,1.71,37.29,XY24,BTT,STEEL
26,GREENPANEL,537.00,232.69,61.0,M-SC,1.82,148572.0,-34506.0,113479.0,1.26,-18.85,76.38,43.14,230.0,-0.30,1.09,39.75,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,QUESS,986.00,-41.60,42.0,M-SC,39.89,57794.0,-7212.0,158142.0,-3.19,-11.09,273.63,232.18,198.0,-0.05,0.42,0.65,XY24,NTT,MISC
80,VAIBHAVGBL,521.00,61.43,64.0,H-SC,6.65,139787.0,-42988.0,154059.0,-2.69,-23.52,110.21,60.77,125.0,-0.28,1.02,25.20,XR,NTT,JEWELLERY
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130899.0,7563.0,126200.0,-2.11,6.13,96.41,108.45,119.0,0.06,0.96,29.86,AR,ATH,MISC
55,RELAXO,1176.00,-33.00,52.0,H-SC,1.91,91330.0,-53830.0,129753.0,-1.94,-37.08,142.07,52.31,136.0,-0.41,0.67,22.25,X40N,NTT,FOOTWEAR
7,ATULAUTO,844.00,3824.32,60.0,M-SC,5.37,122401.0,-24967.0,76782.0,-1.80,-16.94,62.73,35.16,236.0,-0.33,0.90,25.02,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.65,46.0,H-MC,2.18,136606.0,818.0,41938.0,-0.09,0.60,30.70,31.48,107.0,0.02,1.00,12.79,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248882.0,1151.0,73918.0,0.07,0.46,29.70,30.30,92.0,0.02,1.82,13.82,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200400.0,-420.0,69198.0,0.05,-0.21,34.53,34.25,80.0,-0.01,1.47,4.98,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.42,123579.0,-18900.0,38421.0,0.57,-13.27,31.09,13.70,163.0,-0.49,0.90,45.86,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,102.56,52.0,M-SC,3.71,87246.0,-13531.0,13628.0,-1.13,-13.43,15.62,0.09,245.0,-0.99,0.64,14.62,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,63.0,H-SC,1.26,227070.0,-42597.0,78930.0,-0.38,-15.80,34.76,13.47,138.0,-0.54,1.66,14.85,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,44.0,H-SC,2.11,145222.0,-30681.0,72698.0,-0.53,-17.44,50.06,23.89,149.0,-0.42,1.06,25.42,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.45,108462.0,-23373.0,65598.0,-0.44,-17.73,60.48,32.03,98.0,-0.36,0.79,21.49,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.54,233538.0,8576.0,159086.0,1.83,3.81,68.12,74.53,192.0,0.05,1.71,37.29,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.49,196250.0,31580.0,11755.0,0.08,19.18,5.99,26.31,89.0,2.69,1.44,28.69,X40N,NTT,AC
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
17,CAMS,5211.76,0.60,67.0,H-SC,3.47,114307.0,12303.0,31617.0,0.18,12.06,27.66,43.06,122.0,0.39,0.84,31.24,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248882.0,1151.0,73918.0,0.07,0.46,29.70,30.30,92.0,0.02,1.82,13.82,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,64.0,H-MC,7.93,111729.0,11225.0,17877.0,-0.23,11.17,16.00,28.96,79.0,0.63,0.82,48.93,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,0.65,100155.0,8656.0,70098.0,0.00,9.46,69.99,86.07,223.0,0.12,0.73,45.44,XR,NTT,DURABLES
85,WIPRO,420.00,-10.16,57.0,M-LC,5.66,158249.0,7304.0,102150.0,-0.66,4.84,64.55,72.51,53.0,0.07,1.16,11.15,XR,NTT,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130899.0,7563.0,126200.0,-2.11,6.13,96.41,108.45,119.0,0.06,0.96,29.86,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.34,185237.0,5429.0,106974.0,1.20,3.02,57.75,62.52,88.0,0.05,1.36,38.14,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302444.0,-13398.0,126059.0,0.11,-4.24,41.68,35.67,5.0,-0.11,2.21,9.02,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220440.0,-31328.0,84539.0,1.06,-12.44,38.35,21.13,27.0,-0.37,1.61,18.57,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230947.0,-10977.0,74249.0,-1.53,-4.54,32.15,26.15,8.0,-0.15,1.69,9.14,X40,ATH,IT
43,JIOFIN,387.00,0.15,51.0,H-LC,13.14,272104.0,4739.0,60706.0,-0.30,1.77,22.31,24.48,48.0,0.08,1.99,57.72,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-0.29,67.0,H-LC,1.47,247298.0,1147.0,78072.0,-0.52,0.47,31.57,32.18,16.0,0.01,1.81,36.68,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.19,279770.0,14344.0,42441.0,-1.12,5.40,15.17,21.40,11.0,0.34,2.05,13.13,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,58.0,H-LC,4.96,220147.0,9781.0,18999.0,-0.27,4.65,8.63,13.68,37.0,0.51,1.61,21.94,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,53.0,H-LC,5.73,214679.0,10179.0,31236.0,-0.71,4.98,14.55,20.25,10.0,0.33,1.57,14.15,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228399.0,-5745.0,54359.0,0.04,-2.45,23.80,20.77,86.0,-0.11,1.67,4.29,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297049.0,-48907.0,123751.0,-0.52,-14.14,41.66,21.64,1.0,-0.40,2.17,5.23,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302444.0,-13398.0,126059.0,0.11,-4.24,41.68,35.67,5.0,-0.11,2.21,9.02,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230947.0,-10977.0,74249.0,-1.53,-4.54,32.15,26.15,8.0,-0.15,1.69,9.14,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,64.0,H-LC,26.99,166100.0,-19995.0,232391.0,0.13,-10.74,139.91,114.13,15.0,-0.09,1.22,22.32,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,53.0,H-LC,5.73,214679.0,10179.0,31236.0,-0.71,4.98,14.55,20.25,10.0,0.33,1.57,14.15,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220440.0,-31328.0,84539.0,1.06,-12.44,38.35,21.13,27.0,-0.37,1.61,18.57,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,58.0,H-LC,4.96,220147.0,9781.0,18999.0,-0.27,4.65,8.63,13.68,37.0,0.51,1.61,21.94,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297049.0,-48907.0,123751.0,-0.52,-14.14,41.66,21.64,1.0,-0.40,2.17,5.23,X40,ATH,IT
39,INFY,2275.00,-15.65,58.0,H-LC,8.42,328148.0,15260.0,156428.0,1.20,4.88,47.67,54.87,3.0,0.10,2.40,11.85,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302444.0,-13398.0,126059.0,0.11,-4.24,41.68,35.67,5.0,-0.11,2.21,9.02,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,260981.0,-638.0,119842.0,0.02,-0.24,45.92,45.57,7.0,-0.01,1.91,11.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7044.44,66.0,L-SC,5.21,78372.0,-15438.0,92056.0,1.04,-16.46,117.46,81.67,269.0,-0.17,0.57,51.77,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.28,85248.0,-28301.0,68352.0,1.00,-24.92,80.18,35.27,268.0,-0.41,0.62,108.65,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85494.0,-16716.0,110108.0,2.37,-16.35,128.79,91.37,208.0,-0.15,0.63,38.78,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.67,47.0,H-SC,12.49,88739.0,-12224.0,106753.0,-0.56,-12.11,120.30,93.63,148.0,-0.11,0.65,32.17,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94500.0,-3480.0,25080.0,0.29,-3.55,26.54,22.05,152.0,-0.14,0.69,37.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1108.93,70.0,M-SC,4.60,164504.0,-16639.0,62100.0,0.88,-9.19,37.75,25.10,235.0,-0.27,1.20,31.83,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-14.99,78.0,H-SC,12.43,227618.0,-43143.0,373089.0,6.32,-15.93,163.91,121.86,133.0,-0.12,1.67,78.06,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85494.0,-16716.0,110108.0,2.37,-16.35,128.79,91.37,208.0,-0.15,0.63,38.78,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3824.32,60.0,M-SC,5.37,122401.0,-24967.0,76782.0,-1.80,-16.94,62.73,35.16,236.0,-0.33,0.90,25.02,XY24,NTT,AUTO
74,TITAGARH,1548.00,0.74,68.0,H-SC,3.17,165279.0,-26736.0,105613.0,-0.22,-13.92,63.90,41.08,158.0,-0.25,1.21,40.05,XY24,NTT,ENGINEERING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.07
1,25,43.99
2,50,74.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.52
LC    32.11
MC    23.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.33
X40      15.39
X40N     11.94
XR       11.77
AR        8.84
XY25      8.26
OX40N     8.19
X200      1.81
MH        1.70
X5K       1.45
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.38
H-MC    20.32
M-SC    13.63
M-LC     5.66
M-MC     2.71
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.71,-12.18,73.34
FMCG,11.90,-4.08,40.50
FINANCE,9.75,-15.53,60.78
MISC,7.43,-13.35,76.36
BANKS,6.38,-14.65,74.96
PAINTS,5.86,-15.12,32.28
ELECTRICAL,5.76,-4.63,62.29
AC,3.72,4.69,11.27
INSURANCE,3.44,-6.91,43.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2835653.0,22
XR,1300556.0,14
AR,1290711.0,9
X40,744807.0,10
X40N,573454.0,9
OX40N,525936.0,10
XY25,336821.0,6
SR,245476.0,2
X5K,120153.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407086.0,29
M-SC,1382505.0,17
H-LC,1223640.0,14
H-MC,1091868.0,15
M-LC,377506.0,4
M-MC,307492.0,2
L-SC,244072.0,3
L-MC,60391.0,1
L-LC,38622.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1025159.0      6
M-SC       XY24       800061.0      7
H-SC       AR         777350.0      5
           XR         755751.0      7
H-MC       XY24       487032.0      4
H-LC       X40        459535.0      5
           AR         352233.0      2
M-MC       XY24       307492.0      2
H-SC       X40N       288362.0      4
           OX40N      251322.0      4
           SR         245476.0      2
H-MC       X40        214298.0      4
H-LC       X40N       211654.0      3
M-SC       AR         161128.0      2
L-SC       XR         160408.0      2
M-LC       XY24       155203.0      2
H-MC       XY25       126651.0      2
M-SC       OX40N      125352.0      4
M-LC       X5K        120153.0      1
M-SC       XR         114882.0      2
           XY25       110108.0      1
H-MC       XR         106974.0      1
M-LC       XR         102150.0      1
L-SC       OX40N       83664.0      1
H-LC       X200        78072.0      1
H-MC       X40N        73438.0      2
M-SC       X40         70974.0      1
H-MC       OX40N       65598.0      1
H-SC       MH          63666.0      1
H-LC       XY25        61440.0      2
           XY24        60706.0      1
L-MC       XR          60391.0      1
L-LC       XY25        38622.0      1
H-MC       MH          17877.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
